In [36]:
! pip --quite transformers datasets torch torchvision segmentation-models-pytorch albumentations evaluate datasets


Usage:   
  pip3 <command> [options]

no such option: --quite


In [4]:
import os
from google.colab import drive
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# Define path to your ZIP file in Google Drive
zip_path = '/content/drive/Shareddrives/CS198-Drones/Multispectral/D2.zip'

# Extract the ZIP file
extract_path = '/content/dataset'
if not os.path.exists(extract_path):
    os.makedirs(extract_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerForSemanticSegmentation, SegformerConfig
import torchvision.transforms as T
import rasterio
import os
import numpy as np
from torch.nn import functional as F

BASE_DIR =  os.path.join(extract_path, "D2")

# Custom Dataset
class RiceLeafDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = os.path.join(BASE_DIR, image_dir)
        self.label_dir = os.path.join(BASE_DIR, label_dir)
        self.image_files = sorted(os.listdir(self.image_dir))
        self.label_files = sorted(os.listdir(self.label_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        label_path = os.path.join(self.label_dir, self.label_files[idx])

        with rasterio.open(image_path) as img:
            image = img.read().astype(np.float32) / 255.0  # Normalize

        with rasterio.open(label_path) as lbl:
            label = lbl.read(1).astype(np.int64)  # Load single-channel mask

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label, dtype=torch.long)

        return image, label

# Transformations
transform = T.Compose([
    torch.tensor,
])

# Data Loaders
train_dataset = RiceLeafDataset("train", "train_labels", transform=transform)
val_dataset = RiceLeafDataset("val", "val_labels", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Modify SegFormer Model for 6-Band Input
config = SegformerConfig.from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512")
config.num_channels = 6  # Modify input channels
config.num_labels = 5  # 5 classes
model = SegformerForSemanticSegmentation(config)

# Update First Conv Layer
with torch.no_grad():
    old_weight = model.segformer.encoder.patch_embeddings[0].proj.weight
    new_weight = torch.nn.Parameter(torch.randn((64, 6, 7, 7)))  # Adjust for 6 channels
    new_weight[:, :old_weight.shape[1], :, :] = old_weight
    model.segformer.encoder.patch_embeddings[0].proj.weight = new_weight

# Loss, Optimizer, and Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(pixel_values=images).logits

        # Resize the model outputs to match the label size using interpolation
        outputs = F.interpolate(outputs, size=labels.shape[1:], mode='bilinear', align_corners=False)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

print("Training Complete!")


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Epoch 1, Loss: 0.2790884543553086


KeyboardInterrupt: 

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from transformers import SegformerForSemanticSegmentation, SegformerConfig, TrainingArguments, Trainer
import torchvision.transforms as T
import rasterio
import os
import numpy as np
from evaluate import load

# Base Directory
BASE_DIR =  os.path.join(extract_path, "D2")

# Custom Dataset
class RiceLeafDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = os.path.join(BASE_DIR, image_dir)
        self.label_dir = os.path.join(BASE_DIR, label_dir)
        self.image_files = sorted(os.listdir(self.image_dir))
        self.label_files = sorted(os.listdir(self.label_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        label_path = os.path.join(self.label_dir, self.label_files[idx])

        with rasterio.open(image_path) as img:
            image = img.read().astype(np.float32) / 255.0  # Normalize

        with rasterio.open(label_path) as lbl:
            label = lbl.read(1).astype(np.int64)  # Load single-channel mask

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label, dtype=torch.long)

        return {"pixel_values": image, "labels": label}

# Transformations
transform = T.Compose([
    torch.tensor,
])

# Datasets
train_dataset = RiceLeafDataset("train", "train_labels", transform=transform)
val_dataset = RiceLeafDataset("val", "val_labels", transform=transform)

# Modify SegFormer Model for 6-Band Input
config = SegformerConfig.from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512")
config.num_channels = 6  # Modify input channels
config.num_labels = 5  # 5 classes
model = SegformerForSemanticSegmentation(config)

# Update First Conv Layer
with torch.no_grad():
    old_weight = model.segformer.encoder.patch_embeddings[0].proj.weight
    new_weight = torch.nn.Parameter(torch.randn((64, 6, 7, 7)))  # Adjust for 6 channels
    new_weight = torch.nn.Parameter(torch.randn((64, 6, 7, 7)))  # Adjust for 6 channels
    model.segformer.encoder.patch_embeddings[0].proj.weight = new_weight

# Load Accuracy Metric
metric = load("mean_iou")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)  # Convert logits to class indices

    labels = torch.tensor(labels)  # Convert labels to tensor for resizing

    # Resize labels to match predictions (64x64)
    labels = F.interpolate(labels.unsqueeze(1).float(), size=(64, 64), mode="nearest").squeeze(1).long()
    labels = labels.numpy()  # Convert back to NumPy

    # Compute IoU and Accuracy metrics
    metrics = metric.compute(predictions=predictions, references=labels, num_labels=5, ignore_index=0)

    # Replace NaN values with 0
    metrics = {k: (0 if np.isnan(v).any() else v) for k, v in metrics.items()}

    return metrics




# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,  # Ensure logs appear
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the Model
trainer.train()

print("Training Complete!")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
1,0.195800,0.232605,0.408745,0.571458,0.907251,[0.00000000e+00 8.09479943e-01 3.65617345e-05 2.62465165e-01 9.71745246e-01],0


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


TypeError: Object of type ndarray is not JSON serializable